## 任务描述  
<br/>  

**本次实践使用Python来爬取豆瓣网中《速度与激情10》下所有的评论，进行了一波分析，从观众的角度来了解这部电视剧。**  

数据获取：https://movie.douban.com/subject/26631790/reviews  

![Image Name](https://cdn.kesci.com/upload/rvm17lthg3.jpg?imageView2/0/w/960/h/960)  


<br/>  

**上网的全过程:**  

    普通用户:  

    打开浏览器 --> 往目标站点发送请求 --> 接收响应数据 --> 渲染到页面上。  

    爬虫程序:  

    模拟浏览器 --> 往目标站点发送请求 --> 接收响应数据 --> 提取有用的数据 --> 保存到本地/数据库。  


**爬虫的过程**：  

    1.发送请求（requests模块）  

    2.获取响应数据（服务器返回）  

    3.解析并提取数据（BeautifulSoup查找或者re正则）  

    4.保存数据  




**PS:如果爬取不到数据或者遇到报错,可能是由于遇到了服务器端反爬虫机制,需要更换目标网址或根据不同的反爬措施采取不同的解决办法.下面列出几种常见的反爬问题及解决办法:**  

1.UserAgent被屏蔽  
	可收集整理常见的UserAgent以供使用或使用第三方库--fake_useragent.  

2.IP被屏蔽  
	使用代理IP,网上有很多免费代理和付费代理可供选择，除此之外，我们还可以建一个属于自己的代理池以供使用.  

3.Referer防盗链  
	在请求头headers中添加Referer字段以及相应的值。  



<br/>  

**本实践中将会使用以下两个模块，首先对这两个模块简单了解以下：**

<br/>  

**request模块：**  

    requests是python实现的简单易用的HTTP库，官网地址：http://cn.python-requests.org/zh_CN/latest/  
    
    requests.get(url)可以发送一个http get请求，返回服务器响应内容。  
    
    
<br/>  

**BeautifulSoup库：**  

    BeautifulSoup 是一个可以从HTML或XML文件中提取数据的Python库。网址：https://beautifulsoup.readthedocs.io/zh_CN/v4.4.0/  
    
    BeautifulSoup支持Python标准库中的HTML解析器,还支持一些第三方的解析器,其中一个是 lxml。  
    
    BeautifulSoup(markup, "html.parser")或者BeautifulSoup(markup, "lxml")，推荐使用lxml作为解析器,因为效率更高。

# 一、数据爬取

### 1.1、爬取豆瓣网中《速度与激情10》下所有的评论  
发送请求返回页面数据

In [4]:
import json
import re
import requests
from bs4 import BeautifulSoup


def crawl_data(crawl_url):
    '''
    爬取豆瓣《速度与激情10》影评信息，返回response,并进行解析
    '''
    headers = {
        'user-agent':
            "'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'"
        ,
        'cookie':
            'bid=IvtJHpetnDo; __yadk_uid=DFV1ZTyivH9ZbPq3MFJQUVWZcBNYM566; ll="108258"; __utmz=30149280.1686793317.4.3.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __utmz=223695111.1686793321.3.2.utmcsr=douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; _pk_id.100001.4cf6=5737f89bf3526cf1.1681452459.; _vwo_uuid_v2=D0141D5CCE68AEB41C766B408768FB3A4|3b438585d5e7d6f65d2af226ce626fe1; __gads=ID=9bdc14de8e1a2959-2240c51394de002a:T=1681452459:RT=1686793345:S=ALNI_MZD1_aD7LqfrUOMeMqYvydFFGhHJw; __gpi=UID=00000bf4641d88a5:T=1681452459:RT=1686793345:S=ALNI_MYKd2nmI437u9xtmQy4ACXQW7xktg; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1687173941%2C%22https%3A%2F%2Fwww.douban.com%2F%22%5D; _pk_ses.100001.4cf6=1; __utma=30149280.573424069.1681452459.1686798082.1687173942.6; __utmb=30149280.0.10.1687173942; __utmc=30149280; __utma=223695111.255182283.1681452459.1686798082.1687173942.5; __utmb=223695111.0.10.1687173942; __utmc=223695111; ap_v=0,6.0; ucf_uid=553f38e2-4761-4e22-ae3e-c9629f76ee0d; _pbjs_userid_consent_data=3524755945110770; cto_bundle=mu-kkF9UM2glMkZrbjhuN3JPamMlMkJOMDdsb0w0ckVCektyVGRwczUzeTJPVUN3QW5CbFdkVG5VY2NST255OUtFcmZqODNDTEVlWlhkUGxyd21ITTE1YkR6ZU5EQVdQWG9ZN05ZemlCbWU0dlNYZXhSSDZVU1FGeTg4UlJRZG9FZENTWnR4QXBnMmdrcG80WE94ZEluQkdMSTIwQXpBJTNEJTNE; cto_bidid=ryCfeV9Fc2U5b1gyVlFUSFB4ZkRZZTVoRTlaY3dNVUpQQ2k1aGZ4Q0xMUUlSZFZKejhEUUVCU1lXdWxoUzFZSEdoJTJCcG0xaWJQbWQ1cVlicG9QTWRURkVRUmlSM0glMkIlMkJyWmZBWUNKUCUyRmFWdHJDemlRJTNE'
    }
    
    url = 'https://movie.douban.com/subject/26631790/reviews'+crawl_url
    try:
        response = requests.get(url, headers=headers)
        #print(response.status_code)     
        parse(response)
    except Exception as e:
        print(e)

### 1.2、对爬取的页面数据进行解析  
并保存为JSON文件

In [5]:

item_list = []  
authors=[]
pub_times=[]
ratings=[]
titles=[]
contents=[]

def parse(response):
    '''
    从豆瓣返回的html中解析得到选手信息，存JSON文件,保存到work目录下
    '''
    item = {}
    # 将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
    soup = BeautifulSoup(response.text, 'lxml')

    # 返回的是class为“main review-item”的所有<div>标签
    review_list = soup.find_all('div', {'class': 'main review-item'})

    for review_div in review_list:
        # 作者
        author = review_div.find('a', {'class': 'name'}).text
        author = str_format(author)
        # 发布时间
        pub_time = review_div.find('span', {'class': 'main-meta'}).text
        # 评分
        rating = review_div.find('span', {'class': 'main-title-rating'})
        if rating:
            rating = rating.get('title')
        else:
            rating = ""
        # 标题
        title = review_div.find('div', {'class': 'main-bd'}).find('a').text

        # 是否有展开按钮
        is_unfold = review_div.find('a', {'class': 'unfold'})
        if is_unfold:
            # 获取评论id
            review_id = review_div.find('div', {'class': 'review-short'}).get('data-rid')
            # 根据评论id，获取被折叠的评论内容
            content = get_fold_content(review_id)
        else:
            content = review_div.find('div', {'class': 'short-content'}).text
        if content:
            content = re.sub(r"\s", '', content)

        item = {
            "author":author,
            "pub_time":pub_time,
            "rating":rating,
            "title":title,
            "content":content
        }
        # print(item)
        item_list.append(item)
        authors.append(author)
        pub_times.append(pub_time)
        ratings.append(rating)
        titles.append(title)
        contents.append(content)
    
    # 如果有下一页
    next_url = soup.find('span', {'class': 'next'}).find('a')

    if next_url:
        # 请求下一页的数据
        crawl_data(next_url.get('href'))
    else:
        pass




def get_fold_content(review_id):
    '''
    根据评论id，获取被折叠的评论内容
    '''
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
    }
    url = "https://movie.douban.com/j/review/{}/full".format(review_id)

    resp = requests.get(url,headers=headers)
    data = resp.json()

    content = data['html']
    content = re.sub(r"(<.+?>)","",content)

    #去除content中双引号和单引号
    content = str_format(content)
    return content
    

def str_format(line):
    '''
    去除content中特殊字符，单引号、双引号、反斜杠
    '''
    error_list = ['\'','\"','\\',',']
    for c in line:
        if  c in error_list:
            line=line.replace(c,'')
    return line
    

if __name__=='__main__':
    start_url = '?sort=time&start=0'

    crawl_data(start_url)

    #将所爬取的数据，保存为JSON文件
    json_data = json.loads(str(item_list).replace("\'","\"").replace("\\",""))
    with open('./reviews.json', 'a', encoding='UTF-8') as f:        
        json.dump(json_data, f, ensure_ascii=False)
    print(item_list[:5])
    print("爬取完成，共爬取%d条数据"% len(item_list))

[{'author': '犹存', 'pub_time': '2023-06-19 07:00:55', 'rating': '推荐', 'title': '速度与激情10', 'content': '推荐指数：⭐⭐⭐⭐类型：动作导演：路易斯·来特里尔上映日期：2023.05.19时长：140分钟语种：英语---上一次看这个系列好像是大一的时候，看的貌似是第七部，后面的第八部好像也看了，但是没什么印象了，就记得刚开场的时候四辆汽车从飞机上下来。当时直呼好家伙，看完第十部之后才发现这也是常规操作。现在动作片是跟旅游片和科幻片组了cp吗？还以为只能有幸从柯南这种动画上看得到炸水库，并且从水库上一冲而下的镜头呢。不是......我就感觉挺敢拍的。可以理解拍这种电影成本肯定挺高的，风险也很大，而且各个国家都去，就挺烧钱的。电影票贵是正常的，毕竟两个小时的时长在这儿摆着，人们来电影院看就是想过瘾找刺激的。但是但是但是，多少是有点离谱吧。咱不能为了刺激而不顾规律啊。好吧，为了票房和回本确实能。看电影也有一年半了，突然感觉我自己对自己的喜好有了更深一层的了解。比起疯狂刺激大脑分泌多巴胺快感的电影，更喜欢这种有内涵，能看出门道类别的电影。可以不安静，但是一定要让人觉得有深度。比如《十二怒汉》这种，完全黑白，但是张力十足。在一间屋子里拍出来的电影，纯纯靠表演带动，但是节奏张弛有度，字字珠玑，这就是电影的表现力。还有《控方证人》这种反转反转再反转的电影，也是黑白片，剧情完全能给拉上去也很棒。侦探片的话阿婆的一些小说改编的也是去风光美丽的地方拍摄的，但是主题仍然是剧情本身。这种满世界乱转的，除了让人觉得花钱了物超所值之外，我真没觉得在脑子里留下了什么东西。哦对了，最后竟然还没拍完。怎么炸水库之后还能活下来啊！！！这种吊胃口的操作也并不少见，但放在这里就多少感觉有些恶劣了。最后说一句，意大利语还挺好听的，这种尾音往下压的调调听起来好有趣。总是想起《美丽人生》的那句Buongiornoprincipessa.'}, {'author': '机器の猫', 'pub_time': '2023-06-18 16:25:31', 'rating': '力荐', 'title': '影评+全剧情介绍', 'content': '全片的观影节奏比起看的令人昏昏欲睡的九代要好很多，本片中基本每段小文戏之后

# 二、数据分析  

基于所爬取的《速度与激情10》影评信息，分别从评论时间、评分、评论内容进行数据可视化分析。  


<br/>  

**开始之前，导入必要的python包**  
<br/>

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import jieba
import collections
# import wordcloud

import warnings
warnings.filterwarnings('ignore')

<br/>  

**读取数据，并进行预处理**  

将JSON格式的数据转成DataFrame，毕竟数据分析用DataFrame格式会方便很多。代码如下：  
<br/>

In [ ]:
review_df = pd.DataFrame()
# columns=['author','pub_time','rating','title','content']
review_df['author']=authors
review_df['pub_time']=pub_times
review_df['rating']=ratings
review_df['title']=titles
review_df['content']=contents

review_df.head()

In [ ]:
# review_df.to_csv('./fast_and_furious.csv',encoding='utf_8_sig')

In [ ]:
# # 删除缺失数值
# review_df.dropna(inplace=True)

# 将缺失的评论情况设置为"放弃"
review_df[review_df['rating']=='']['rating'] = '放弃'
review_df[review_df['rating']=='']['rating'] = '放弃'

# # 将字符串格式的时间转换为datatime类型
review_df['pub_time'] = pd.to_datetime(review_df['pub_time'])
review_df.head()

## 2.1、评论数量趋势图  

评论数量随着日期的变化呈现出怎样的趋势。接下来我们就实际动手，来看看随着《速度与激情10》的正式上映，评论数量呈现出怎样的趋势。  

2023年05月17日上映.代码如下：  


In [ ]:
# 分析评论日期 
import re
from matplotlib import dates
#显示matplotlib生成的图形
%matplotlib inline



#添加一个索引“pub_date”，存储评论日期（2020年2月到3月期间）
review_df['pub_date'] = review_df['pub_time'].dt.date

review_df = review_df[pd.to_datetime(review_df['pub_date']).dt.year==2023]
# review_df = review_df[pd.to_datetime(review_df['pub_date']).dt.month>1]
# review_df = review_df[pd.to_datetime(review_df['pub_date']).dt.month<4]

# 根据评论日期进行聚合
review_date_df = review_df['author'].groupby(review_df['pub_date'])
review_date_df = review_date_df.count()

In [ ]:
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.commons.utils import JsCode


x_data = [str(i)[5:] for i in review_date_df.index]
y_data = [int(i) for i in review_date_df.values]
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
)
area_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
)

c = (
    Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="注册总量",
        y_axis=y_data,
        is_smooth=True,
        is_symbol_show=True,
        symbol="circle",
        symbol_size=6,
        linestyle_opts=opts.LineStyleOpts(color="#fff"),
        label_opts=opts.LabelOpts(is_show=True, position="top", color="white"),
        itemstyle_opts=opts.ItemStyleOpts(
            color="red", border_color="#fff", border_width=3
        ),
        tooltip_opts=opts.TooltipOpts(is_show=False),
        areastyle_opts=opts.AreaStyleOpts(color=JsCode(area_color_js), opacity=1),
    )
    .set_series_opts(
                markarea_opts=opts.MarkAreaOpts(
                    data=[
                        opts.MarkAreaItem(x=(s,e)) for s,e in zip([10],[12])
                    ]
                ))
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="《速度与激情10》评论数趋势图",
            pos_bottom="90%",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(color="#fff", font_size=16),
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            boundary_gap=False,
            axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63",rotate=50),
            axisline_opts=opts.AxisLineOpts(is_show=False),
            axistick_opts=opts.AxisTickOpts(
                is_show=True,
                length=25,
                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
            ),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
            ),
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            position="right",
            axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63"),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
            ),
            axistick_opts=opts.AxisTickOpts(
                is_show=True,
                length=15,
                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
            ),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
            ),
        ),
        legend_opts=opts.LegendOpts(is_show=False),
    )
)
c.render_notebook()


## 2.2、评论时间分布图  

分析用户一般在哪些时间段评论得比较多。这里从0点到24点，2个小时为一个时间段统计评论数量，代码如下：

In [ ]:
import datetime

#指定多个时间区间
time_range = [0,2,4,6,8,10,12,14,16,18,20,22,24]

#获取评论时间中的hour
review_time_df = review_df['pub_time'].dt.hour

#把一组数据分割成离散的区间，并获取每个区间的评论数
#第一个参数--->review_time_df：被切分的数据；
#第二个参数---->bins：被切割后的区间
#第三个参数：-->right:表示是否包含区间右部
time_range_counts = pd.cut(review_time_df,bins=time_range,right=False).value_counts()

In [ ]:
from pyecharts.charts import Bar
from pyecharts import options as opts
bar2=(
        Bar()
        .add_xaxis([str(i) for i in time_range_counts.index])
        .add_yaxis("数据1",[int(i) for i in time_range_counts.values])
# category_gap是同一系列的柱间距离，默认为类目间距的 20%，可设固定值
        .set_series_opts(itemstyle_opts={# set_series_opts设置系列配置
            "normal":{ # normal代表一般、正常情况
# LinearGradient 设置线性渐变，offset为0是柱子0%处颜色，为1是100%处颜色
                "color": JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                    offset: 0, 
                    color: 'rgba(0, 233, 245, 1)'
                }, {
                    offset: 1, 
                    color: 'rgba(0, 45, 187, 1)'
                }], false)"""),
                "barBorderRadius": [30, 30, 30, 30],# 设置柱子4个角为30变成圆柱
                "shadowColor": 'red',# 阴影颜色
            }})
        .set_global_opts(title_opts=opts.TitleOpts(title="24小时时间段的发表评论数"))
    )
bar2.render_notebook()

## 2.3、电影评分  
筛选出有评分的评价,并将电影评价映射到数值，进行分析。  
1、总体评分  
2、每日评分均值走向

In [ ]:
print(review_df.shape)
df_score=review_df[review_df['rating']!=''].copy()
df_score.shape

In [ ]:
def map_score(x):
    if x == '力荐':
        return 5
    elif x == '推荐':
        return 4
    elif x == '还行':
        return 3
    elif x == '较差':
        return 2
    elif x == '很差':
        return 1
df_score['score']=df_score['rating'].apply(map_score)
df_score.head()

### 2.3.1、电影总体评分

In [ ]:
df_score['rating'].value_counts()

In [ ]:
from pyecharts.charts import Pie
from pyecharts import options as opts

# 示例数据
cate = [str(i) for i in df_score['rating'].value_counts().index]
data = [int(i) for i in df_score['rating'].value_counts().values]

pie = (Pie()
       .add('', [list(z) for z in zip(cate, data)],
            radius=["30%", "75%"],
            rosetype="radius"
            )
       .set_global_opts(title_opts=opts.TitleOpts(title="《速度与激情10》评分", subtitle="总体分布"))
       .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
      )

pie.render_notebook()

### 2.3.2、每日评分均值趋势图

In [ ]:
from pyecharts.charts import Line
from pyecharts import options as opts

# 示例数据
cate = [str(i)[6:] for i in df_score.groupby(['pub_date'])['score'].mean().index]
data2 = [float(i) for i in df_score.groupby(['pub_date'])['score'].mean().values]

line = (Line()
       .add_xaxis(cate)
       .add_yaxis('每日均分', data2,is_smooth=True)
       .set_global_opts(title_opts=opts.TitleOpts(title="每日评分均值趋势图", subtitle="红色区域为电影上映时间"),
                xaxis_opts=opts.AxisOpts(name='日期',axislabel_opts={"rotate":50}))
       .set_series_opts(
                markarea_opts=opts.MarkAreaOpts(
                    data=[
                        opts.MarkAreaItem(x=(s,e)) for s,e in zip([5.5],[50])
                    ]
                ),
                label_opts=opts.LabelOpts(is_show=False)
      ))

line.render_notebook()

## 2.4、词云图分析  
1、总体词云  
2、正面评论词云  
3、负面评论词云

### 2.4.1、总体评论词云图

In [ ]:
import jieba
stwlist=[line.strip() for line in open('/home/mw/project/停用词库.txt','r',encoding='utf-8').readlines()]
content_all=''
for i in review_df['content']:
    content_all+=i
stwlist.append('\r\n')
dele = set(stwlist)
words = list(jieba.cut(content_all))
articleDict = {}
articleSet = set(words)-dele
for w in articleSet:
    if len(w)>1:
        articleDict[w] = words.count(w)

articlelist = sorted(articleDict.items(),key = lambda x:x[1], reverse = True)

for i in range(10):
    print(articlelist[i])

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType

words = articlelist
c = (
    WordCloud()
    .add("", words, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="《速度与激情10》评论词云图"))
)
c.render_notebook()

### 2.5.2、好评词云

In [ ]:
review_good=df_score[df_score['score']>3]
content_all=''
for i in review_good['content']:
    content_all+=i
stwlist.append('\r\n')
dele = set(stwlist)
words = list(jieba.cut(content_all))
articleDict = {}
articleSet = set(words)-dele
for w in articleSet:
    if len(w)>1:
        articleDict[w] = words.count(w)

articlelist = sorted(articleDict.items(),key = lambda x:x[1], reverse = True)

for i in range(10):
    print(articlelist[i])

words = articlelist
c = (
    WordCloud()
    .add("", words, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="《速度与激情10》好评词云图"))
)
c.render_notebook()

### 2.5.3、差评词云图

In [ ]:
review_bad=df_score[df_score['score']<3]
content_all=''
for i in review_bad['content']:
    content_all+=i
stwlist.append('\r\n')
dele = set(stwlist)
words = list(jieba.cut(content_all))
articleDict = {}
articleSet = set(words)-dele
for w in articleSet:
    if len(w)>1:
        articleDict[w] = words.count(w)

articlelist = sorted(articleDict.items(),key = lambda x:x[1], reverse = True)

for i in range(10):
    print(articlelist[i])

words = articlelist
c = (
    WordCloud()
    .add("", words, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="《速度与激情10》差评词云图"))
)
c.render_notebook()

## 2.5、实体识别  

In [ ]:
# content_all
from nltk.probability import FreqDist

#分词
words = jieba.lcut(content_all, cut_all=False)

#去停用词
with open('/home/mw/project/停用词库.txt') as f:
    stopwords = [line.strip() for line in f.readlines()]
    
words = [w for w in words if w not in stopwords]
words = [w for w in words if w != ' ']

fdist = FreqDist(words)
print("所有词语数量：", fdist.N())
print("数量最多的词：",fdist.max())

In [ ]:
#n 普通名词 f 方位名词 s 处所名词 t 时间 nr 人名 ns 地名 nt 机构名 nw 作品名 nz 其他专名
#v 普通动词 vd 动副词 vn 名动词
#a 形容词 ad 副形词 an 名形词 d 副词
#m 数量词 q 量词 r 代词 p 介词
#c 连词 u 助词 xc 其他虚词 w 标点符号
# PER 人名 LOC 地名 ORG 机构名 TIME 时间


# 按词性找出词语
nouns_tags = ['n','f','s','t','nr','ns','nt','nw','nz']
verbs_tags = ['v','vd','vn']
adjs_tags = ['a','ad','an','d']

import jieba.posseg as psg

words_tagged = list(psg.cut(content_all)) # 附加词性的分词结果

### 2.5.1、名词实体

In [ ]:
from collections import Counter
#名词
nouns=[item.word for item in words_tagged if item.flag in nouns_tags]

d_nouns = Counter(nouns)

df_nouns = pd.DataFrame.from_dict(d_nouns, orient='index').reset_index()
df_nouns = df_nouns.rename(columns={'index':'words', 0:'count'})
df_nouns = df_nouns.sort_values(by='count', ascending=False)

df_nouns.head()

In [ ]:
import pyecharts
counter = [tuple(x) for x in df_nouns.values[:50]]

bar = (Bar(init_opts=opts.InitOpts(theme='purple-passion', width='1000px', height='800px'))
       .add_xaxis([x for x, y in counter[::-1]])
       .add_yaxis('出现次数', [y for x, y in counter[::-1]], category_gap='30%')
       .set_global_opts(title_opts=opts.TitleOpts(title="出现最多的名词",
                                                  pos_left="center",
                                                  title_textstyle_opts=opts.TextStyleOpts(font_size=20)),
                        datazoom_opts=[opts.DataZoomOpts(range_start=70, range_end=100, orient='vertical')],
                        visualmap_opts=opts.VisualMapOpts( max_=106, min_=10, dimension=0,
                                range_color=['#f5d69f', '#f5898b', '#ef5055']),
                        legend_opts=opts.LegendOpts(is_show=False),
                        xaxis_opts=opts.AxisOpts(is_show=False,),
                        yaxis_opts=opts.AxisOpts(axistick_opts=opts.AxisTickOpts(is_show=False),
                                                 axisline_opts=opts.AxisLineOpts(is_show=False)))
       .set_series_opts(label_opts=opts.LabelOpts(is_show=True,
                                                  position='right',
                                                  font_style='italic'),
                        itemstyle_opts={"normal": {
                                                    "barBorderRadius": [30, 30, 30, 30],
                                                    'shadowBlur': 10,
                                                    'shadowColor': 'rgba(120, 36, 50, 0.5)',
                                                    'shadowOffsetY': 5,
                                                }
                                       }
).reversal_axis())

bar.render_notebook()

## 2.6、关键词识别